<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Lab: Cleaning Rock Song Data

_Authors: Dave Yerrington (SF)_

---


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [2]:
rockfile = "datasets/rock.csv"

In [3]:
# Load the data.

df = pd.read_csv(rockfile)
df.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [4]:
# Look at the information regarding its columns.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
Song Clean      2230 non-null object
ARTIST CLEAN    2230 non-null object
Release Year    1653 non-null object
COMBINED        2230 non-null object
First?          2230 non-null int64
Year?           2230 non-null int64
PlayCount       2230 non-null int64
F*G             2230 non-null int64
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [5]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
    Read a comma-separated values (csv) file into DataFrame.
    
    Also supports option

In [6]:
# Change the column names when loading the '.csv':
df = pd.read_csv(rockfile, 
                 names=['Song', 'Artist', 'Release Year', 'Combined', 'First', 'Year', 'Playcount', 'Total Plays'])
df.head()

,Song,Artist,Release Year,Combined,First,Year,Playcount,Total Plays
0,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
1,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
2,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
3,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
4,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [11]:
# Change the column names using the `.rename()` function.
df = pd.read_csv(rockfile)

df = df.rename(columns={
    'Song Clean': 'song',
    'ARTIST CLEAN': 'artist',
    'Release Year': 'release_year',
    'COMBINED': 'combined_title',
    'First?': 'first_place',
    'Year?': 'year',
    'PlayCount':'play_count',
    'F*G':'year_by_play_count'
})
df.head()

#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [14]:
# Replace the column names by reassigning the `.columns` attribute.
df = pd.read_csv(rockfile)
df.columns = ['song', 'artist', 'release_year', 'combined_title', 'first_place', 'year', 'play_count', 'year_by_play_count']
df.head()

,song,artist,release_year,combined_title,first_place,year,play_count,year_by_play_count
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [18]:
# Show records where df['release'] is null
release = df['release_year']
df[release.isnull()].head()

,song,artist,release_year,combined_title,first_place,year,play_count,year_by_play_count
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0


### 4. Update slices of your DataFrame based on mask selection/slices.

In many scenarios, we want to upate values in our DataFrame according to criteria. Let's say we wanted to set all of the null values in `release` to 0.

With newer versions of `pandas`, in order to manipulate data in the original DataFrame, we have to use `.loc` while performing reassignment using a mask and an index.

For example, the following won't always work:
```python
df[row_mask]['column_name'] = new_value
```

The best way to accomplish the same task is:
```python
df.loc[row_mask, 'column_name'] = new_value
```

For multiple column assignment, you would use:
```python
df.loc[row_mask, ['col_1', 'col_2', 'col_3']] = new_value
```

#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [20]:
# Replace release nulls with 0
df.loc[release.isnull(), 'release_year'] = 0

#### 4.B Verify that `release` contains no null values.

In [22]:
# A:
df[release.isnull()].head()

,song,artist,release_year,combined_title,first_place,year,play_count,year_by_play_count


### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

In [24]:
df.head()

,song,artist,release_year,combined_title,first_place,year,play_count,year_by_play_count
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


In [23]:
# A:
df.info()

# Release year could be an integer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
song                  2230 non-null object
artist                2230 non-null object
release_year          2230 non-null object
combined_title        2230 non-null object
first_place           2230 non-null int64
year                  2230 non-null int64
play_count            2230 non-null int64
year_by_play_count    2230 non-null int64
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [27]:
# A:
df['release_year'].astype('int64')

# The below error shows that a string 'SONGFACTS.COM' is being stored in the release column

ValueError: invalid literal for int() with base 10: 'SONGFACTS.COM'

#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [28]:
# A:
for item in df['release_year']:
    try:
        int(item)
    except:
        print(item)

SONGFACTS.COM


In [29]:
df.loc[df['release_year'] == 'SONGFACTS.COM']

,song,artist,release_year,combined_title,first_place,year,play_count,year_by_play_count
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [30]:
# A:
df.loc[df['release_year'] == 'SONGFACTS.COM', 'release_year'] = 0
df['release_year'] = df['release_year'].astype('int64')

### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [33]:
# A:

df['release_year'].describe()

count    2230.000000
mean     1465.331390
std       867.196161
min         0.000000
25%         0.000000
50%      1973.000000
75%      1981.000000
max      2014.000000
Name: release_year, dtype: float64

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

In [35]:
# A:
# The earliest non-zero release year is 1071
df.loc[df['release_year'] !=0].release_year.describe()

count    1652.000000
mean     1978.019976
std        24.191247
min      1071.000000
25%      1971.000000
50%      1977.000000
75%      1984.000000
max      2014.000000
Name: release_year, dtype: float64

_Looking at the DataFrame that contains the year 1071, we can see that the year was probably corrupted and should be replaced with something else if possible._

### 8. Make changes and investigate using custom functions with `.apply()`.

Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.


In [43]:
# A:

# For a single row:
def show_info(row):
        song = row['song']
        artist = row['artist']
        released_1970 = row['release_year'] < 1970
        print(song, artist, released_1970)
        
# Test it out
show_info(df.loc[0])

Caught Up in You .38 Special False


#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [44]:
# A:
df.apply(show_info, axis=1)

Caught Up in You .38 Special False
Fantasy Girl .38 Special True
Hold On Loosely .38 Special False
Rockin' Into the Night .38 Special False
Art For Arts Sake 10cc False
Kryptonite 3 Doors Down False
Loser 3 Doors Down False
When I'm Gone 3 Doors Down False
What's Up? 4 Non Blondes False
Take On Me a-ha False
Baby, Please Don't Go AC/DC True
Back In Black AC/DC False
Big Gun AC/DC False
CAN'T STOP ROCK'N'ROLL AC/DC True
Dirty Deeds Done Dirt Cheap AC/DC False
For Those About To Rock AC/DC False
Girls Got Rhythm AC/DC True
Hard As A Rock AC/DC False
Have a Drink On Me AC/DC False
Hells Bells AC/DC False
Highway To Hell AC/DC False
It's A Long Way To The Top AC/DC False
Jailbreak AC/DC False
Let There Be Rock AC/DC False
Let's Get It Up AC/DC True
Live Wire AC/DC True
Moneytalks AC/DC True
Night Prowler AC/DC False
Rock and Roll Ain't Noise Pollution AC/DC False
Shoot To Thrill AC/DC True
Shot Down In Flames AC/DC False
Sin City AC/DC True
T.N.T. AC/DC False
Thunderstruck AC/DC False
Touc

Barracuda Heart False
Bebe le Strange Heart True
Crazy On You Heart False
Dog & Butterfly Heart True
Dreamboat Annie Heart False
Even It Up Heart True
Heartless Heart True
How Can I Refuse Heart False
If Looks Could Kill Heart True
Kick It Out Heart True
Little Queen Heart True
Love Alive Heart True
Magic Man Heart False
Mistral Wind Heart True
Never Heart True
Nothin' at All Heart True
Rock And Roll Heart True
Stairway To Heaven Heart True
Straight On Heart False
Tell It Like It Is Heart True
What About Love? Heart True
Who Will You Run To Heart True
House Of Living Heartsfield True
Sangre Por Sangre HellYeah True
Grey Ghost Henry Paul Band False
Long Cool Woman Hollies False
New Girl Now Honeymoon Suite True
ALL YOU ZOMBIES Hooters False
AND WE DANCED Hooters False
I Want A New Drug Huey Lewis & The News False
30 Days In the Hole Humble Pie False
Lust for Life Iggy Pop False
The Passenger Iggy Pop False
Radioactive Imagine Dragons False
Devil Inside INXS False
Need You Tonight INXS F

Words As Weapons Seether True
God Save The Queen Sex Pistols False
No Turning Back Sherbs True
45 Shinedown False
America Simon & Garfunkel True
Cecilia Simon & Garfunkel False
Mrs. Robinson Simon & Garfunkel True
The Sounds Of Silence Simon & Garfunkel True
Don't You Simple Minds False
18 And Life Skid Row False
I Remember You Skid Row False
Youth Gone Wild Skid Row False
Fly To The Angels Slaughter False
All Star Smash Mouth False
1979 Smashing Pumpkins False
Bullet With Butterfly Wings Smashing Pumpkins False
Landslide Smashing Pumpkins True
Today Smashing Pumpkins False
Driver's Seat Sniff 'n' the Tears False
Tainted Love Soft Cell False
Black Hole Sun Soundgarden False
Burden in My Hand Soundgarden False
Fell On Black Days Soundgarden False
Spoonman Soundgarden False
Havin' A Party Southside Johnny True
The Fever Southside Johnny True
GIMME SOME LOVIN' Spencer Davis Group True
Two Princes Spin Doctors False
Black Coffee In Bed Squeeze False
Pulling Mussels Squeeze False
Tempted Sq

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2200    None
2201    None
2202    None
2203    None
2204    None
2205    None
2206    None
2207    None
2208    None
2209    None
2210    None
2211    None
2212    None
2213    None
2214    None
2215    None
2216    None
2217    None
2218    None
2219    None
2220    None
2221    None
2222    None
2223    None
2224    None
2225    None
2226    None
2227    None
2228    None
2229    None
Length: 2230, dtype: object

You'll notice that there will be a final output Series of `None` values. The `.apply()` function, if a return value is not specified, will return a Series of `None` values (similar to how the default return for Python functions is `None` when a return statement is not specified).

### 9. Write a function that converts cells in a DataFrame to float and otherwise replaces them with `np.nan`.

If applied to our data, it would keep only the numeric information and otherwise input null values.

Recall that the try-except syntax in Python is a great way to try something and take another action if the initial step fails:

```python
try:
    Perform some action.
except:
   Perform some other action if the first failed with an error.
```

#### 9.A Write the function that takes a column and converts all of its values to float if possible and `np.nan` otherwise. The return value should be the converted Series.

In [51]:
# A:
def convert_floats(column):
    new_column = []
    for item in column:
        try:
            x = float(item)
        except:
            x = np.nan
        new_column.append(x)
    return pd.Series(new_column)

#### 9.B Try your function out on the rock song data and ensure the output is what you expected.


In [52]:
# A:
# Test it out
print(convert_floats(df['play_count']).head())
print(convert_floats(df['artist']).head())

0    82.0
1     3.0
2    85.0
3    18.0
4     1.0
dtype: float64
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64


#### 9.C Describe the new float-only DataFrame.

In [53]:
# A:
df.apply(convert_floats, axis=0).describe()

,song,artist,release_year,combined_title,first_place,year,play_count,year_by_play_count
count,2.000000,0.0,2230.000000,0.0,2230.0,2230.000000,2230.000000,2230.000000
mean,1012.000000,NaN,1465.331390,NaN,1.0,0.741256,16.872646,15.048430
std,1367.544515,NaN,867.196161,NaN,0.0,0.438043,25.302972,25.288366
min,45.000000,NaN,0.000000,NaN,1.0,0.000000,0.000000,0.000000
25%,528.500000,NaN,0.000000,NaN,1.0,0.000000,1.000000,0.000000
50%,1012.000000,NaN,1973.000000,NaN,1.0,1.000000,4.000000,3.000000
75%,1495.500000,NaN,1981.000000,NaN,1.0,1.000000,21.000000,18.000000
max,1979.000000,NaN,2014.000000,NaN,1.0,1.000000,142.000000,142.000000
